In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from IPython.display import Image

# --- State class ---
class State(TypedDict):
    message: str

# --- Node functions ---
def start_fn(state):
    print("➡️ Start Node")
    state["message"] = "Hello from Start"
    return state

def next_fn(state):
    print("➡️ Next Node, got:", state["message"])
    return state

# --- Build graph ---
builder = StateGraph(State)
builder.add_node("start_node", start_fn)
builder.add_node("next_node", next_fn)

builder.set_entry_point("start_node")
builder.add_edge("start_node", "next_node")
builder.add_edge("next_node", END)

graph = builder.compile()

# Run
graph.invoke({"message": ""})

# Show graph
#graph_png = graph.get_graph().draw_mermaid_png()
#display(Image(graph_png))



➡️ Start Node
➡️ Next Node, got: Hello from Start


{'message': 'Hello from Start'}

In [2]:
%pip install -U langgraph langchain langchain-openai

  Obtaining dependency information for langgraph from https://files.pythonhosted.org/packages/67/06/f440922a58204dbfd10f7fdda0de0325529a159e9dc3d1038afe4b431a49/langgraph-0.6.7-py3-none-any.whl.metadata
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/f6/d5/4861816a95b2f6993f1360cfb605aacb015506ee2090433a71de9cca8477/langchain-0.3.27-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-openai from https://files.pythonhosted.org/packages/67/31/af0486b7ad8a49f3c5c852ca2b3a7f6d8526cc71a405045dd959c36ec5db/langchain_openai-0.3.33-py3-none-any.whl.metadata
  Obtaining dependency information for langgraph-checkpoint<3.0.0,>=2.1.0 from https://files.pythonhosted.org/packages/4c/dd/64686797b0927fb18b290044be12ae9d4df01670dce6bb2498d5ab65cb24/langgraph_checkpoint-2.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for langgraph-prebuilt<0.7.0,>=0.6.0 from https://files.pythonhosted.org/packages/0a/7f/973b0d9729d96

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.1.2 requires chromadb<0.6.0,>=0.4.0, but you have chromadb 1.0.13 which is incompatible.
langchain-chroma 0.1.2 requires langchain-core<0.3,>=0.1.40, but you have langchain-core 0.3.76 which is incompatible.
langchain-chroma 0.1.2 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.3.1 which is incompatible.


In [4]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    num: int

def check_fn(state):
    print("Checking number:", state["num"])
    return state

def even_fn(state):
    print(" Even number")
    return state

def odd_fn(state):
    print(" Odd number")
    return state

def route_decider(state):
    return "even_label" if state["num"] % 2 == 0 else "odd_label"

builder = StateGraph(State)
builder.add_node("check_node", check_fn)
builder.add_node("even_node", even_fn)
builder.add_node("odd_node", odd_fn)

builder.set_entry_point("check_node")
builder.add_conditional_edges("check_node", route_decider, {"even_label": "even_node", "odd_label": "odd_node"})
builder.add_edge("even_node", END)
builder.add_edge("odd_node", END)

graph = builder.compile()
graph.invoke({"num": 6})



Checking number: 6
 Even number


{'num': 6}

In [7]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    count: int

def increment(state):
    state["count"] += 1
    print("count =", state["count"])
    return state

def stop_condition(state: State) -> str:
    return "continue_again" if state["count"] < 5 else "complete_done"

builder = StateGraph(State)
builder.add_node("increment_node", increment)

builder.set_entry_point("increment_node")
builder.add_conditional_edges(
    "increment_node",
    stop_condition,
    {"continue_again": "increment_node", "complete_done": END}
)

graph = builder.compile()
graph.invoke({"count": 0})




count = 1
count = 2
count = 3
count = 4
count = 5


{'count': 5}

In [5]:
from langgraph.graph import StateGraph, END

def step1(state):
    print("Step 1: Add 2")
    state["x"] += 2
    return state

def step2(state):
    print("Step 2: Multiply by 3")
    state["x"] *= 3
    return state

def step3(state):
    print("Step 3: Subtract 5")
    state["x"] -= 5
    return state

builder = StateGraph(dict)
builder.add_node("s1_node", step1)
builder.add_node("s2_node", step2)
builder.add_node("s3_node", step3)

builder.set_entry_point("s1_node")
builder.add_edge("s1_node", "s2_node")
builder.add_edge("s2_node", "s3_node")
builder.add_edge("s3_node", END)

graph = builder.compile()
graph.invoke({"x": 4})


Step 1: Add 2
Step 2: Multiply by 3
Step 3: Subtract 5


{'x': 13}